# Đọc/ghi file Parquet trong Hadoop sử dụng mô hình MapReduce

* Sử dụng thư viện Apache Spark trên EMR để tiến hành đọc file .csv từ Amazon S3 và chuyển thành file .parquet và cũng dùng chính thư viện này để tiến hành đọc/ghi file .parquet.

##### 1. Kiểm tra kết nối giữa Cluster của EMR và Instance của SageMaker Notebook

In [ ]:
%%info

##### 2. Tiến hành đọc file .csv từ S3 và hiển thị kết quả, đồng thời cho biến thời gian để lấy thời gian đọc

In [ ]:
//Biến tính thời gian
var t = System.nanoTime

//Tạo biến df chứa dataframe của file .csv đọc được từ S3
val df = spark.read.option("header",true).csv("s3://hadoopbucket177103/input/train.csv")

//In thời gian thực thi câu lệnh trên, convert thời gian từ Nanosec sang Sec, chuyển về double
var duration = (System.nanoTime - t) / 1e9d

//In kết quả có được sau khi đọc file .csv (20 dòng đầu tiên)
df.show()

#### 3. Ghi Dataframe vào file .csv lưu trữ trên Amazon S3, đồng thời xác định thời gian để thực thi

In [ ]:
t = System.nanoTime

//Ghi Dataframe vào file .csv
df.write.mode("overwrite").csv("s3://hadoopbucket177103/output/train.csv")

duration = (System.nanoTime - t) / 1e9d

##### 4. Ghi Dataframe vào file .parquet lưu trữ trên Amazon S3, đồng thời xác định thời gian để thực thi

In [ ]:
t = System.nanoTime

//Ghi Dataframe vào file .parquet
df.write.mode("overwrite").parquet("s3://hadoopbucket177103/output/train.parquet")

duration = (System.nanoTime - t) / 1e9d

##### 5. Đọc file .parquet trên Amazon S3 và đưa vào Dataframe, đồng thời xác định thời gian để thực thi

In [ ]:
t = System.nanoTime

//Tạo biến df2 chứa dataframe của file .parquet đọc được từ S3
val df2 = spark.read.parquet("s3://hadoopbucket177103/output/train.parquet")

duration = (System.nanoTime - t) / 1e9d

//Hiển thị kết quả lấy được từ file .parquet trong Dataframe
df2.show();

### 6. Thực hiện 1 số câu query trên Parquet

In [ ]:
//Tạo tempView để thực hiện truy vấn SQL
df2.createOrReplaceTempView("ParquetTable")

//Thực hiện câu truy vấn
t = System.nanoTime
val parkSQL = spark.sql("select * from ParquetTable where user_answer > 2 ")
duration = (System.nanoTime - t) / 1e9d

//show table
parkSQL.show(10)

In [ ]:
//Thực hiện câu truy vấn
t = System.nanoTime
val parkSQL = spark.sql("select * from ParquetTable where (prior_question_elapsed_time >= 5000) and (prior_question_elapsed_time <= 15000) ")
duration = (System.nanoTime - t) / 1e9d

//show table
parkSQL.show(10)